# Apply to financial applications

Present notebook will use the present library to develop a specific case for financial application. The reference for this notebook will be:

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

The idea is given an input function $f(x)$ and a proability density $p(x)$ is compute the expected value of  $f(x)$ when x follows $p(x)$:

$$E_{x\sim p}(f)=\int_a^bp(x)f(x)dx$$

In this case we are going to use the discretized form:

$$E_{x\sim p}(f) = \sum_{x=0}^{2^n-1} p(x)f(x)dx$$

The idea is to use **Amplitude Amplification** procedure in order to compute this value.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
import sys
import qat.lang.AQASM as qlm
sys.path.append("../")

In [ ]:
%load_ext qat.core.magic
%matplotlib inline

In [ ]:
global_qlmaas = True
try:
    from qlmaas.qpus import LinAlg
except (ImportError, OSError) as e:
    global_qlmaas = False
from qat.qpus import PyLinalg

In [ ]:
def get_qpu(qlmass=False):
    """
    Function for selecting solver. User can chose between:
    * LinAlg: for submitting jobs to a QLM server
    * PyLinalg: for simulating jobs using myqlm lineal algebra.

    Parameters
    ----------

    qlmass : bool
        If True  try to use QLM as a Service connection to CESGA QLM
        If False PyLinalg simulator will be used

    Returns
    ----------
    
    lineal_qpu : solver for quantum jobs
    """
    if qlmass:
        if global_qlmaas:
            print('Using: LinAlg')
            linalg_qpu = LinAlg()
        else:
            raise ImportError("""Problem Using QLMaaS.Please create config file or use mylm solver""")
            
    else:
        print('Using PyLinalg')
        linalg_qpu = PyLinalg()
    return linalg_qpu

In [ ]:
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg
QLMaaS = True
linalg_qpu = get_qpu(QLMaaS)

## 1. Discretizing data

Below cell defines the functions for $p(x)$ and $f(x)$

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

First thing to do is discretizing these functions. We can use the **get_histogram** function from *utils* module

In [ ]:
from libraries.utils import get_histogram

In [ ]:
#number of Qbits for the circuit
n_qbits = 4

In [ ]:
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

It is mandatory that $p(x)$ and $f(x)$ were properly normalized. Following conditions should be fullfilled:

* For $p(x)$ is mandatory that: $\sum_{i=0}^{2^{n}} p(i) \leq 1$
* For $f(x)$ is mandatory that: $f(i) \leq 1 \forall i$

In [ ]:
%matplotlib inline
plt.plot(X, p_X, 'o')
plt.plot(X, f_X, 'o')
plt.legend(['p(x)', 'f(x)'])

In [ ]:
#Test condition in p(x)
print('p(x) condition: {}'.format(np.sum(p_X) == 1))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

## 2. Loading data

Second part is loading the data ($p(x)$ and $f(x)$) into the quantum state. For this we need to create following 2 loading operators:

1. $\mathcal{P}$. for loading $p(x)$
2. $\mathcal{F}$. for loading $f(x)$

So the main idea is begin with an initial $n+1$ qbits state:

$$|0\rangle_{n}\otimes|0\rangle_{1}$$

An apply the following loading operation:

$$|\Psi\rangle=\mathcal{F}\left(\mathcal{P}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

The quantum state resulting from the loading proccess: $|\Psi\rangle$ can be expressed as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|1\rangle$$
$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|0\rangle$$
and
$$ a = \sum_{n=0}^{2^{n}-1} p(x)f(x)$$

With this operations we have loaded the desired integral into the amplitude of the state: $|\Psi_{0}\rangle$

For doing this operations **load_probability** and **load_array** functions from *data_loading* module will be used (see notebook **01_DataLoading_Module_Use.ipynb**)

In [ ]:
from libraries.data_loading import load_probability, load_array

In [ ]:
p_gate = load_probability(p_X)
f_gate = load_array(np.sqrt(f_X))

In [ ]:
%qatdisplay p_gate --depth 0
%qatdisplay f_gate --depth 0

Now we compound the two operators $\mathcal{P}$ aqnd $\mathcal{F}$ for creating the necesary loading circuit

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(f_gate.arity)
routine.apply(p_gate, register[:p_gate.arity])
routine.apply(f_gate, register)

In [ ]:
%qatdisplay routine --depth 0

We can test if the loading was properly implemented. For doing this we are going to use **get_results** from *data_extracting* module

In [ ]:
from libraries.data_extracting import get_results

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
np.isclose(quantum_probabilities[0:2**p_gate.arity], p_X*f_X).all()

We are going to measure the complete state (**phi_state**) and the last qbit state (**InitialState**)

In [ ]:
phi_state,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)#, qubits=[f_gate.arity-1])
InitialState,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu, qubits=[f_gate.arity-1])

In [ ]:
np.isclose(InitialState['Probability'].iloc[0], sum(p_X*f_X))

## 3. Groover Operator

The main idea of **amplitude amplification** is to amplify the amplitude of the  $|\Psi_{0}\rangle$ for computing the desired integral. For this the Grover-like operator $\mathcal{G}$ will be used. In the present framework this operator can be decomposed in 2 different operators:

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$


In [ ]:
from libraries.AE.amplitude_amplification_bis import U0, U, grover

### 3.1 $\hat{U}_{|\Psi_{0}\rangle} = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$

For constuct this operator we need:
1. Oracle: in our case this will be the complete loading routine:

$$\mathcal{F}\left(\mathcal{P}\otimes I\right)$$

2. Target: the binnary representation of the marked state. 

3. index: the registers where the Target is.

In our case the marked state is $|\Psi_{0}\rangle$:

 $$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|0\rangle$$

This is equivalent to a target state $|0\rangle$ on the last qbit of the quantum sistem!!


In [ ]:
oracle = routine
target = [0]
index = [routine.arity-1]
u0_gate = U0(routine, target, index)

In [ ]:
%qatdisplay u0_gate --depth 0

In [ ]:
u0_routine = deepcopy(routine)
u0_routine.apply(u0_gate, register)

In [ ]:
%qatdisplay u0_routine --depth 0

In [ ]:
results_u0,_,_,_ = get_results(u0_routine, linalg_qpu=linalg_qpu)

In [ ]:
print(np.isclose(
    results_u0[results_u0['Int_lsb'] < 2**n_qbits]['Amplitude'],
    -phi_state[phi_state['Int_lsb'] < 2**n_qbits]['Amplitude']
).all())

print(np.isclose(
    results_u0[results_u0['Int_lsb'] > 2**n_qbits]['Amplitude'],
    phi_state[phi_state['Int_lsb'] > 2**n_qbits]['Amplitude']
).all())

### 3.2 $\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$

In this case we only need to provide the Oracle routine for creating the Groover diffusor

In [ ]:
u_gate = U(routine)
%qatdisplay u_gate --depth 1

In [ ]:
u_routine = deepcopy(routine)
u_routine.apply(u_gate, register)

In [ ]:
%qatdisplay u_routine --depth 0

In [ ]:
results_u,_,_,_ = get_results(u_routine, linalg_qpu=linalg_qpu)

In [ ]:
np.isclose(
    results_u['Amplitude'], 
    -phi_state['Amplitude']
).all()

### 3.3 $\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$

For creating the desired Grover-like operator we should provide the same that for the $\hat{U}_{|\Psi_{0}\rangle}$ operator:

For constuct this operator we need:
1. Oracle: in our case this will be the complete loading routine:

$$\mathcal{F}\left(\mathcal{P}\otimes I\right)$$

2. Target: the binnary representation of the marked state. 

3. index: the registers where the Target is.

In our case the marked state is $|\Psi_{0}\rangle$:

 $$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|0\rangle$$

This is equivalent to a target state $|0\rangle$ on the last qbit of the quantum sistem!!

In [ ]:
oracle = routine
target = [0]
index = [routine.arity-1]
q_gate = grover(oracle, target, index)
%qatdisplay q_gate --depth 2

In [ ]:
q_routine = deepcopy(routine)
q_routine.apply(q_gate, register)

In [ ]:
%qatdisplay q_routine --depth 0

In [ ]:
results_q,_,_,_ = get_results(q_routine, linalg_qpu=linalg_qpu, qubits=[u_routine.arity-1])

In [ ]:
#First get the Amplitudes for Phi state
a0 = np.sqrt(InitialState.iloc[0]['Probability'])
a1 = np.sqrt(InitialState.iloc[1]['Probability'])

In [ ]:
#Calculating Theta using the quantum state from loading data: LResults
def GetAngle(Array):
    Modulo = np.linalg.norm(Array)
    cosTheta = Array[0]/Modulo
    Theta0 = np.arccos(cosTheta)
    sinTheta = Array[1]/Modulo
    Theta1 = np.arcsin(sinTheta)
    #print(Modulo, cosTheta, sinTheta, Theta0, Theta1)
    return Theta0

#Quantum state after loading data: |Psi>
Psi_ = np.array([a0, a1])
#Angle between |Psi> and axis |Psi_0>
theta = GetAngle(Psi_)
print('theta: {}'.format(theta))
print('Psi_: {}'.format(Psi_))

In [ ]:
#In order to test that this theta is ok we can compute it from p(x) and f(x)
a = sum(p_X*f_X)
sqrt_a = np.sqrt(a)
theta2 = np.arccos(sqrt_a)
print('theta2: {}'.format(theta2))

print('Is theta equals to theta2: {}'.format(np.isclose(theta, theta2)))

In [ ]:
#Create a Rotation of 2*theta
c, s = np.cos(2*theta), np.sin(2.*theta)
#Rotation matrix
R = np.array(((c, -s), (s, c)))
#Apply Ry(2*theta) to quantum state |Psi>
RotationResults = np.dot(R, Psi_)

In [ ]:
RotationResults

In [ ]:
print('Square Of Rotated Vector: {}'.format(RotationResults**2))
print('Probabilities for QPhi_State: {} '.format(list(results_q['Probability'])))
print('Square of the RotateState equal to Probabilities of Q|Phi> state : {}'.format(
    np.isclose(RotationResults**2, results_q['Probability']).all())
     )

## 4. Phase Estimation with Quantum Fourier Transformation

Now we have the initial state after the loading procedure:

$$|\Psi\rangle=\mathcal{F}\left(\mathcal{P}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

That can be decomposed in:

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

And the corresponding Grover-like operator $\mathcal{Q}$ based on the loading operator:

$$\mathcal{F}\left(\mathcal{P}\otimes I\right)$$

The main idea in the **amplitude amplification** is use a similar procedure like in Grover algorithm in order to increase the probability of the $|\Psi_{0}\rangle$ state. This can be in a straightoforward way by applying operator $\mathcal{Q}$ $k$ times:

$$\hat{Q}^{k}|\Psi\rangle = \sin{((2*k+1)\theta)}|\Psi_{1}\rangle+\cos{((2*k+1)\theta})|\Psi_{0}\rangle$$

The idea is apply $k$ in such a way that the probability of $|\Psi_{0}\rangle$ is maximized. For this we can equal:


$$P_{|\Psi_{1}\rangle} = \cos^{2}((2*k+1)\theta) \approx 1$$ 

So:

$$(2k+1)\theta = m\pi$$ where $m=0, 1, 2...$

We have two unknows variables: $\theta$ and $K$. If we know $\theta$ we can know how many applications of $\hat{Q}$ we need for maximazing the state we want.

The **Phase Estimation Algorithm** can be used for estimating this $\theta$:

For this we need to create n auxiliary qbits in state $|+\rangle$. Each qbit will be the controlled qbit for a controlled application of $\mathcal{Q}^{K}$ (for each controlled qbit the K will change). Finally over the auxiliary qbits we apply an inverse of the Quantum Fourier Operator ($\mathcal{QFT}$). The measurmente of this auxiliary qbits will give us an estimation of the desired $\theta$. This estimation will be more exact when more auxiliary qbits we use for the $\mathcal{QFT}$.

For this algorithm if we have $m$ auxiliary qbits and the measurment of the $\mathcal{QFT}$ is the integer $M$ then:

$$\theta = \frac{M\pi}{2^m}$$

In [ ]:
def load_qn_gate(qlm_gate, n_times):
    """
    Create an AbstractGate by applying an input gate n times

    Parameters
    ----------

    qlm_gate : QLM gate
        QLM gate that will be applied n times
    n_times : int
        number of times the qlm_gate will be applied

    """
    @qlm.build_gate("Q^{}".format(n_times), [], arity=qlm_gate.arity)
    def q_n_gate():
        """
        Function generator for creating an AbstractGate for apply
        an input gate n times
        Returns
        ----------
        q_rout : quantum routine
            Routine for applying n times an input gate
        """
        q_rout = qlm.QRoutine()
        q_bits = q_rout.new_wires(qlm_gate.arity)
        for _ in range(n_times):
            q_rout.apply(qlm_gate, q_bits)
        return q_rout
    return q_n_gate()


In [ ]:
#Allocating Auxiliar qbits for Phase Estimation
n_aux = 7
qft_routine = deepcopy(routine)
qAux = qft_routine.new_wires(n_aux)
for i, aux in enumerate(qAux):
    qft_routine.apply(qlm.H, aux)
    step_q_gate = load_qn_gate(q_gate, 2**i)
    qft_routine.apply(step_q_gate.ctrl(), aux, register)
qft_routine.apply(qlm.qftarith.QFT(n_aux).dag(), qAux)    

In [ ]:
%%time
results_q,circuit,_,_ = get_results(
    qft_routine, 
    linalg_qpu=linalg_qpu,
    qubits=range(routine.arity, routine.arity+len(qAux))
)

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
results_q['Thetas'] = results_q['Int']*np.pi/(2**n_aux)
results_q.sort_values('Probability', ascending=False, inplace = True)

In [ ]:
results_q.head()

In [ ]:
len(results_q)

In [ ]:
#Now we can plot te probability for each posible theta
%matplotlib inline
plt.plot(results_q['Thetas'], results_q['Probability'], 'o')
plt.xlabel('Theta')
plt.ylabel('Probability')

In [ ]:
#We take the thetas were probability is maximum
theta_0 = results_q.sort_values('Probability', ascending =False)['Thetas'].iloc[0]
theta_1 = results_q.sort_values('Probability', ascending =False)['Thetas'].iloc[1]

print('theta_0: {}'.format(theta_0))
print('theta_1: {}'.format(theta_1))

In [ ]:
#So we test that the 2 are the same angle with different signs
np.isclose(theta_0, np.pi -theta_1)

In [ ]:
#Now we can plot te probability for each posible theta
%matplotlib inline
plt.plot(np.cos(results_q['Thetas'])**2, results_q['Probability'], 'o')
plt.xlabel('E_p[f]')
plt.ylabel('Probability')

In [ ]:
theoric_theta = np.arccos(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))
print('theta_1: {}'.format(theta_0))
print('Theorical Integration: {}'.format(sum(p_X*f_X)))
print('Amplitude Amplification Integral: {}'.format(np.cos(theta_0)**2))

In [ ]:
theoric_theta = np.arccos(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))
print('theta_1: {}'.format(theta_0))
print('Theorical Integration: {}'.format(sum(p_X*f_X)))
print('Amplitude Amplification Integral: {}'.format(np.cos(theta_0)**2))
print('Estimation Difference: {}'.format(abs(sum(p_X*f_X)-np.cos(theta_0)**2)))

## 5. Maximum Likelihood Amplitude Estimation

Quantum circuit implementation of $\mathcal{QFT}$ are very complex and very long and deep so its use in actual quantum computers is noisy and not very useful.

The **Maximum Likelihood Amplitude Estimation** algorithm (**MLAE**) allows to take advantage of **amplitude amplification** without the use of the  $\mathcal{QFT}$.

For doing this  we are going to use the  **maximum_likelihood_ae** module 

In [ ]:
from  libraries.AE.maximum_likelihood_ae  import MLAE

The notebook **04_MaximumLikelihood_Class** explains how this class works. In our case we need to provide:

1. Oracle
2. Target: the binnary representation of the marked state. 
3. index: the registers where the Target is.

In our case is the same that we use for generating Grover-like operator

In [ ]:
oracle = deepcopy(routine)
target = [0]
index = [routine.arity-1]

In [ ]:
%qatdisplay oracle

Additionally a schedule where the list of applications of Grover-like operator should be provided

In [ ]:
m_k = list(range(20))
n_k = [100]*len(m_k)
schedule = [m_k,n_k]

In [ ]:
mlae = MLAE(
    oracle,target = target,
    index = index,
    schedule = schedule)

In [ ]:
angle = mlae.optimize()

In [ ]:
print("Theta: {}".format(angle))
print("Quantum result: ",np.sin(angle)**2)

In [ ]:
sum(p_X*f_X)

In [ ]:
delta = 1e-5
angles = np.linspace(0+delta,np.pi/2-delta,100)

cost = np.zeros(len(angles))
for i in range(len(angles)):
    cost[i] = mlae.cost_function(angles[i])

In [ ]:
plt.plot(angles,cost)
#plt.axvline(theta,color = 'r')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")

## 6. Iterative

Additionally we can use the *Iterative Phase Estimation* for getting the $\theta$ for this problem.
For using **IterativeQuantumPE** we need to provide:

* initial_state : in this case will be the oracle
* unitary_operator:  in this case will be the Grover-like operator build using *oracle* 
* cbits_number: number of classical bits used for phase estimation

For creating the grover operator we are going to use the *grover* function from *amplitude_amplification* module and the same *target^and *index*  then in the *maximum likelihood* class will be used:

In [ ]:
from libraries.AE.amplitude_amplification_bis import grover

In [ ]:
target = [0]
index = [oracle.arity-1]
q_gate = grover(oracle, target, index)

In [ ]:
%qatdisplay q_gate --depth 1

In [ ]:
#Load Class
from libraries.PE.iterative_quantum_pe_2 import IterativeQuantumPE

In [ ]:
n_cbits = 7
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': q_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    'shots': 100
    #'easy': True,
    #'easy': False    
}
IQPE = IterativeQuantumPE(**iqpe_dict)

In [ ]:
IQPE.iqpe()

In [ ]:
plt.hist(IQPE.final_results['theta_90'], bins= 100)

In [ ]:
pdf = IQPE.sumary
pdf['P']=np.cos(pdf['theta_90'])**2

In [ ]:
print("Classical result: ", np.sum(p_X*f_X))
print("Quantum result: ",pdf.iloc[0]['P'])